# 🧠 Legal Assistant – Jordanian Labour Law Notebook

## 📌 Overview
This notebook implements a **Legal Assistant chatbot** focused on the **Jordanian Labour Law (Law No. 8 of 1996)**.  
The system is designed to ingest legal articles, clean and validate them, chunk the text, and prepare it for **LLM-based question answering** in a safe and structured manner.

⚠️ **Disclaimer**  
This system generates **informational legal drafts and summaries only**.  
It does **NOT provide legal advice** and does **NOT replace a licensed lawyer**.  
Any generated contract, answer, or explanation is a **non-binding draft** and must be reviewed by a qualified legal professional before use.

---

## 🎯 Objectives
- Convert raw labour law text into structured JSON
- Validate article integrity and detect domain-level gaps
- Chunk legal text for retrieval-augmented generation (RAG)
- Prepare clean and traceable inputs for a legal chatbot
- Ensure every chunk can be traced back to its legal source

---

## 🧱 Notebook Pipeline

### 1️⃣ Data Ingestion
The notebook loads the Jordanian Labour Law articles from a structured JSON file.  
Each record contains an `article_number` and the full legal `text`.

Example:
```json
{
  "article_number": "15",
  "text": "Article (15): Employment Contract Preparation..."
}
2️⃣ Text Cleaning
Minimal preprocessing is applied to preserve legal meaning:
1_Normalize whitespace and line breaks
2_Remove OCR artifacts and duplicated spacing
3_Preserve original legal wording (no paraphrasing)
Example:
Before:
Article (15):   Employment   Contract   Preparation
After:
Article (15): Employment Contract Preparation


3️⃣ Chunking Strategy
Long legal articles are split into smaller semantic chunks to improve:

1_Retrieval accuracy
2_Context window efficiency for LLMs
3_Precise legal referencing
*Each chunk includes*:

1_article_number
2_chunk_id
3_text
Example:
{
  "article_number": "15",
  "chunk_id": "15_2",
  "text": "If the work contract is for a definite period..."
}

4️⃣ Validation & Sanity Checks
A sanity report is automatically generated to validate dataset quality:

1_Total number of extracted articles
2_Detected gaps in article numbering due to legal amendments
3_Chunk size distribution
Data consistency checks
The output is saved as:
sanity_report.json
This ensures the dataset reflects real legislative structure, not artificial completeness.
5️⃣ LLM Readiness
The final processed dataset is optimized for:
Legal question-answering chatbots
Retrieval-Augmented Generation (RAG)
Draft contract and legal explanation generation
Example chatbot query:
“Can an employer terminate a limited employment contract early?”
The system retrieves the relevant article(s) and generates an answer with explicit legal grounding.
🚀 End Goal

To build a safe, explainable, and auditable Legal Assistant system that:
Understands Jordanian Labour Law
Provides grounded answers with article references
Produces draft-only legal outputs with clear disclaimers
Supports future export to Word / PDF contract templates

In [ ]:
!pip install PyPDF2



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from pathlib import Path
from PyPDF2 import PdfReader
import re
import json

PDF_PATH = Path(
    r"C:\Users\VICTUS-H\Desktop\Faris\Datasets\raw\jordanian_labour_law_no._(8)_of_1996_and_its_amendments.PDF"
)

reader = PdfReader(PDF_PATH)

full_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        full_text += text + "\n"

print("TEXT LENGTH:", len(full_text))
print("-" * 50)
print(full_text[:2000])


TEXT LENGTH: 118964
--------------------------------------------------
1 
 Jordanian Labour Law No. (8) Of 1996 and its Amendments 
 
 
CHAPTER ONE  
Scope of Application of this Law  and Interpretation of Terms  
 
Article (1):  Title and Effective Date  
This Law shall be cited as the "The labour code for the year 1996" and shall come 
into effect after sixty days of its publication in the Official Gazette.  
 
Article (2):  Definitions Wherever used in this Law, and unless the context 
otherwise provides, the following terms and expressions, shall have the meaning 
hereunder assigned to them:  
The Ministry : The Ministry of Labour.  
The Minister:  The Minister of Labour.  
Secretary General:  Secretary General of the Ministry.  
The Employer:  Every natural or co rporate person engaging, in any capacity, a 
person or more in return for a wage.  
The Emp loyers Association:  The committee representing Employers.  
The Employee:  Every person, wheth er male or female, who performs 

In [16]:
def clean_text(text: str) -> str:
    text = text.replace("", "")
    text = text.replace("\u00a0", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()


In [17]:
article_pattern = re.compile(
    r"(?:^|\s)(Article)\s*(?:\(?\s*(\d+)\s*\)?)(?:\s*[:\-])?",
    re.IGNORECASE
)

articles = []
current_article = None

for line in full_text.splitlines():
    article_match = article_pattern.search(line)

    if article_match:
        if current_article:
            articles.append(current_article)

        current_article = {
            "article_number": article_match.group(2),
            "text": clean_text(line) + "\n"
        }
    else:
        if current_article:
            current_article["text"] += clean_text(line) + "\n"

if current_article:
    articles.append(current_article)

print(f"Extracted {len(articles)} articles")
print("SAMPLE ARTICLE:")
print(articles[0])


Extracted 155 articles
SAMPLE ARTICLE:
{'article_number': '1', 'text': 'Article (1): Title and Effective Date\nThis Law shall be cited as the "The labour code for the year 1996" and shall come\ninto effect after sixty days of its publication in the Official Gazette.\n\n'}


In [18]:
article_numbers = [int(a["article_number"]) for a in articles]

print("First 10 articles:", article_numbers[:10])
print("Last 10 articles:", article_numbers[-10:])

missing_articles = sorted(
    set(range(min(article_numbers), max(article_numbers) + 1)) - set(article_numbers)
)

print("Missing articles:", missing_articles)


First 10 articles: [1, 2, 43, 56, 3, 4, 5, 6, 7, 8]
Last 10 articles: [133, 134, 135, 136, 137, 138, 139, 140, 141, 142]
Missing articles: [18]


In [19]:
with open("labour_law_articles.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, ensure_ascii=False, indent=2)


In [20]:
law_metadata = {
    "law_name": "Jordanian Labour Law",
    "law_number": "8",
    "year": 1996,
    "jurisdiction": "Jordan",
    "language": "en",
    "source": "Official Gazette"
}

normalized_articles = []

for art in articles:
    normalized_articles.append({
        **law_metadata,
        "article_number": art["article_number"],
        "text": clean_text(art["text"])
    })

len(normalized_articles)


155

In [21]:
def chunk_text(text, max_chars=2000, overlap=300):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_chars
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


In [22]:
chunks = []

for art in normalized_articles:
    text_chunks = chunk_text(art["text"])
    for i, chunk in enumerate(text_chunks):
        chunk_id = f"JL8-1996-A{art['article_number']}-C{i}"

        chunks.append({
            "chunk_id": chunk_id,
            "law_name": art["law_name"],
            "article_number": art["article_number"],
            "chunk_index": i,
            "text": clean_text(chunk)
        })

len(chunks)


165

In [23]:
with open("labour_law_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)


In [24]:
import json

article_numbers = [int(a["article_number"]) for a in articles]

missing_articles = sorted(
    set(range(min(article_numbers), max(article_numbers) + 1)) - set(article_numbers)
)

sanity_report = {
    "dataset_info": {
        "law_name": "Jordanian Labour Law",
        "law_number": 8,
        "year": 1996,
        "language": "English",
        "source": "Official Gazette (with amendments)"
    },
    "statistics": {
        "total_characters": len(full_text),
        "total_articles_extracted": len(articles),
        "total_chunks_created": len(chunks),
        "article_numbers_range": {
            "min": min(article_numbers),
            "max": max(article_numbers)
        }
    },
    "article_length": {
        "min": min(len(a["text"]) for a in articles),
        "max": max(len(a["text"]) for a in articles),
        "avg": sum(len(a["text"]) for a in articles) // len(articles)
    },
    "chunk_length": {
        "min": min(len(c["text"]) for c in chunks),
        "max": max(len(c["text"]) for c in chunks),
        "avg": sum(len(c["text"]) for c in chunks) // len(chunks)
    },
    "missing_articles_analysis": {
        "count": len(missing_articles),
        "articles": missing_articles,
        "explanation": (
            "Some article numbers are missing due to official legal amendments, "
            "repealed provisions, or article mergers in later versions of the law. "
            "This is a domain-specific characteristic of legislative documents "
            "and not a data extraction or parsing error."
        )
    },
    "data_quality_status": "PASSED_WITH_DOMAIN_GAPS"
}

with open("sanity_report.json", "w", encoding="utf-8") as f:
    json.dump(sanity_report, f, indent=2)

sanity_report


{'dataset_info': {'law_name': 'Jordanian Labour Law',
  'law_number': 8,
  'year': 1996,
  'language': 'English',
  'source': 'Official Gazette (with amendments)'},
 'statistics': {'total_characters': 118964,
  'total_articles_extracted': 155,
  'total_chunks_created': 165,
  'article_numbers_range': {'min': 1, 'max': 142}},
 'article_length': {'min': 44, 'max': 4692, 'avg': 747},
 'chunk_length': {'min': 2, 'max': 2000, 'avg': 715},
 'missing_articles_analysis': {'count': 1,
  'articles': [18],
  'explanation': 'Some article numbers are missing due to official legal amendments, repealed provisions, or article mergers in later versions of the law. This is a domain-specific characteristic of legislative documents and not a data extraction or parsing error.'},
 'data_quality_status': 'PASSED_WITH_DOMAIN_GAPS'}